In [ ]:
import os
import keras as keras
from keras.layers import Dense
import tensorflow as tf
from keras.optimizers import Adam
import gym
import numpy as np

In [ ]:
!pip install --upgrade tensorflow-probability==0.23.0

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import tensorflow_probability as tfp

/usr/local/lib/python3.10/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <


In [ ]:
class ActorCriticNetwork(keras.Model):
  def __init__(self, n_actions, fc1_dims=1024, fc2_dims=512,
               name='actor_critic', chkpt_dir='tmp/actor_critic'):
    super(ActorCriticNetwork, self).__init__()
    self.fc1_dims = fc1_dims
    self.fc2_dims = fc2_dims
    self.n_actoions = n_actions
    self.model_name=name
    self.checkpoint_dir = chkpt_dir
    self.checkpoint_file = os.path.join(self.checkpoint_dir, name+'_ac')

    self.fc1 = Dense(self.fc1_dims, activation='relu')
    self.fc2 = Dense(self.fc2_dims, activation='relu')

    self.v = Dense(1, activation=None)
    self.pi = Dense(n_actions, activation='softmax')

  def call(self, state):
    value = self.fc1(state)
    value = self.fc2(value)

    v = self.v(value)
    pi = self.pi(value)

    return v, pi

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
class Agent:
  def __init__(self, alpha=0.0003, gamma=0.99, n_actions=2):
    self.gamma = gamma
    self.n_actions = n_actions
    self.action = None
    self.action_space = [i for i in range(self.n_actions)]

    self.actor_critic = ActorCriticNetwork(n_actions=n_actions)

    self.actor_critic.compile(optimizer=Adam(learning_rate=alpha))

  def choose_action(self, observation):
    state = tf.convert_to_tensor([observation]) # add one dimension
    _, probs = self.actor_critic(state)

    action_probabilities = tfp.distributions.Categorical(probs=probs)
    action = action_probabilities.sample()
    self.action = action

    return action.numpy()[0]

  def save_models(self):
    print('... saving models ...')
    self.actor_critic.save_weights(self.actor_critic.checkpoint_file)

  def load_model(self):
    self.actor_critic.load_weights(self.actor_critick.checkpoint_file)

  def learn(self, state, reward, state_, done):
    state = tf.convert_to_tensor([state], dtype=tf.float32)
    state_ = tf.convert_to_tensor([state_], dtype=tf.float32)
    reward = tf.convert_to_tensor(reward, dtype=tf.float32)

    with tf.GradientTape(persistent=True) as tape:
      state_value, probs = self.actor_critic(state)
      state_value_, _ = self.actor_critic(state_)
      state_value = tf.squeeze(state_value)
      state_value_ = tf.squeeze(state_value)

      action_probs = tfp.distributions.Categorical(probs=probs)
      log_prob = action_probs.log_prob(self.action)

      delta = reward + self.gamma*state_value_*(1-int(done)) - state_value
      actor_loss = -log_prob*delta
      critic_loss = delta**2
      total_loss = actor_loss + critic_loss

    gradient = tape.gradient(total_loss, self.actor_critic.trainable_variables)
    self.actor_critic.optimizer.apply_gradients(zip(
        gradient, self.actor_critic.trainable_variables
    ))

In [ ]:
env = gym.make('CartPole-v0')
agent = Agent(alpha=1e-5, n_actions=env.action_space.n)
n_games = 1800

filename='carpole.png'

figure_file = 'plots/' + filename

best_score = env.reward_range[0]
score_history = []
load_checkpoint = False

if load_checkpoint:
  agent.load_models()

for i in range(n_games):
  observation = env.reset()
  done = False
  score = 0
  while not done:
    action = agent.choose_action(observation)
    observation_, reward, done, info = env.step(action)
    score += reward
    if not load_checkpoint:
      agent.learn(observation, reward, observation_, done)
    observation = observation_
    score_history.append(score)
    avg_score = np.mean(score_history[-100:])

    if avg_score > best_score:
      best_score = avg_score
      if not load_checkpoint:
        agent.save_models()
    print('episode ', i, 'score %.1f' % score, 'avg_score %.1f' % avg_score)

A saída de streaming foi truncada nas últimas 5000 linhas.
episode  1437 score 9.0 avg_score 8.3
episode  1438 score 1.0 avg_score 8.2
episode  1438 score 2.0 avg_score 8.0
episode  1438 score 3.0 avg_score 7.9
episode  1438 score 4.0 avg_score 7.7
episode  1438 score 5.0 avg_score 7.6
episode  1438 score 6.0 avg_score 7.4
episode  1438 score 7.0 avg_score 7.5
episode  1438 score 8.0 avg_score 7.5
episode  1438 score 9.0 avg_score 7.6
episode  1438 score 10.0 avg_score 7.7
episode  1438 score 11.0 avg_score 7.7
episode  1438 score 12.0 avg_score 7.8
episode  1438 score 13.0 avg_score 7.8
episode  1438 score 14.0 avg_score 7.9
episode  1438 score 15.0 avg_score 8.0
episode  1438 score 16.0 avg_score 8.0
episode  1438 score 17.0 avg_score 8.1
episode  1439 score 1.0 avg_score 8.0
episode  1439 score 2.0 avg_score 7.8
episode  1439 score 3.0 avg_score 7.7
episode  1439 score 4.0 avg_score 7.8
episode  1439 score 5.0 avg_score 7.8
episode  1439 score 6.0 avg_score 7.8
episode  1439 score 7

In [ ]:
!mkdir plots tmp tmp/actor_critic